In [9]:
import scanpy as sc
import numpy as np
import pickle
import umap
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [10]:
#load adata
#adata = sc.read_h5ad('TS.h5ad')
#pickle.dump(adata.X, open('TS_X.pickle', 'wb'))
#pickle.dump(adata.obs['free_annotation'], open('cell_labels.pickle', 'wb'))

In [11]:
f = open('TS_X.pickle', 'rb')
matrix = pickle.load(f)

In [41]:
f = open('cell_labels.pickle', 'rb')
labels = pickle.load(f)

In [12]:
f.close()

In [15]:
matrix[0,:].todense()

matrix([[0.       , 0.       , 0.       , ..., 4.6512003, 0.       ,
         0.       ]], dtype=float32)

In [5]:
shape = matrix.shape

In [8]:
# index = np.arange(100000)
# np.random.shuffle(index)
# matrix = matrix[index, :]

In [9]:
import gc
gc.collect()

474

In [10]:
#matrix = matrix.todense()

In [11]:
#matrix = np.asarray(matrix)

In [38]:
import struct
with open('full_matrix.dat', 'wb') as f:
    for row in matrix:
        # IO is slow but I think this helps by putting like 58,000 things in one syscall
        # Still takes like 5 mintues or something
        f.write(struct.pack('f'*matrix.shape[1], *list(np.asarray(row.todense())[0,])))

In [23]:
f.close()

In [31]:
matrix = None
gc.collect()

1926

In [37]:
with open('cell_labels.pickle', 'rb') as f:
    labels = pickle.load(f)
f.close()

In [45]:
labels = np.array(labels).tobytes()

In [46]:
with open('cell_labels.dat', 'wb') as f:
    f.write(labels)

In [40]:
with open('shape.dat', 'wb') as f:
    f.write(struct.pack('ii', shape[0], shape[1]))

In [39]:
shape

(483152, 58482)